In [1]:
pip install torch

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
pip install itertools

Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement itertools (from versions: none)
ERROR: No matching distribution found for itertools

[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
import pandas as pd
from tqdm import tqdm
import random
import os
from datetime import datetime
import numpy as np
import torch
import torch.nn as nn
import itertools
# from google.colab import drive
# drive.mount('/gdrive')

In [4]:
# finance-datareader(주식 데이터)/pandas_market_calendars(주말, 공휴일 제거용) 다운로드
!pip install -U finance-datareader
!pip install pandas_market_calendars


[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [5]:
pip install -U finance-datareader

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [6]:
pip install plotly


[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [7]:
import FinanceDataReader as fdr
import pandas_market_calendars as mcal
import random

# 한국거래소 전체 상장종목
df_krx = fdr.StockListing('KRX')
df_krx.head()

,Code,ISU_CD,Name,Market,Dept,Close,ChangeCode,Changes,ChagesRatio,Open,High,Low,Volume,Amount,Marcap,Stocks,MarketId
0,005930,KR7005930003,삼성전자,KOSPI,,57300,2,-300,-0.52,57600,58000,56300,22092218,1262148460900,342068540115000,5969782550,STK
1,000660,KR7000660001,SK하이닉스,KOSPI,,195800,1,2600,1.35,197700,199200,192700,3460361,678932664000,142542863067000,728002365,STK
2,373220,KR7373220003,LG에너지솔루션,KOSPI,,390500,2,-29500,-7.02,426500,429000,386000,689223,276337181500,91377000000000,234000000,STK
3,207940,KR7207940008,삼성바이오로직스,KOSPI,,1026000,1,18000,1.79,1013000,1040000,998000,103863,105452978000,73024524000000,71174000,STK
4,005380,KR7005380001,현대차,KOSPI,,206500,2,-8500,-3.95,214000,214500,205500,1880618,393383186500,43244443441500,209416191,STK


In [8]:
df_krx["Name"]

0           삼성전자
1         SK하이닉스
2       LG에너지솔루션
3       삼성바이오로직스
4            현대차
          ...   
2840        씨알푸드
2841      씨앤에스링크
2842     티엘엔지니어링
2843       나라소프트
2844         테크엔
Name: Name, Length: 2845, dtype: object

사용자로부터 종목 이름, 날짜 입력받아 각각 일별 등락률 가져오기

In [9]:
# 사용자로부터 종목 이름 입력 받기
selected_companies_names = []
for i in range(3):
    company_name = input(f"종목 이름을 입력하세요 {i+1}: ")
    selected_companies_names.append(company_name)

# 사용자로부터 날짜 입력 받기
input_date = input("형식에 맞춰 날짜를 입력하세요. (YYYY-MM-DD): ")
start_date = pd.to_datetime(input_date) - pd.DateOffset(years=1)

# 종목 이름을 종목 코드로 변환하고 등락률을 저장할 데이터프레임 초기화
df_returns = pd.DataFrame()
df_close_price = pd.DataFrame()


# 각 종목에 대해 데이터 불러오고 등락률 가져오기
for company_name in selected_companies_names:
    # 종목 코드 가져오기
    try:
        company_code = df_krx[df_krx['Name'] == company_name]['Code'].values[0]
        df_stock = fdr.DataReader(company_code, start_date, input_date)
        df_returns[company_name] = df_stock['Change']
        df_close_price[company_name] = df_stock['Close']
    except IndexError:
        print(f"입력하신 {company_name}은 존재하지 않습니다.")

df_returns

종목 이름을 입력하세요 1:  삼성전자
종목 이름을 입력하세요 2:  현대차
종목 이름을 입력하세요 3:  카카오
형식에 맞춰 날짜를 입력하세요. (YYYY-MM-DD):  20241106


,삼성전자,현대차,카카오
Date,,,
2023-11-06,0.018678,0.025952,0.082324
2023-11-07,0.000000,-0.010118,-0.021253
2023-11-08,-0.014104,-0.007382,0.004571
2023-11-09,0.005722,-0.000572,0.037543
2023-11-10,0.002845,-0.005724,0.001096
...,...,...,...
2024-10-31,0.001692,-0.024943,-0.010710
2024-11-01,-0.015203,-0.006977,-0.021651
2024-11-04,0.006861,0.014052,0.033195


위에서 가져온 일별 등락률을 바탕으로 <br>
개별 주식 평균 등락률 E(r1), E(r2), E(r3) 구하기<br>
E(r) : 개별 주식의 일별 등락률 평균 x 날짜(일 수)

In [10]:
daily_stock_average_d = []
average_returns = df_returns.mean()
average_returns = average_returns * len(df_returns)
for value in average_returns:
    daily_stock_average_d.append(value)

print(average_returns)
print("daily_stock_average_d:", daily_stock_average_d)

삼성전자   -0.153791
현대차     0.243521
카카오    -0.065994
dtype: float64
daily_stock_average_d: [-0.1537911898681571, 0.24352137918975625, -0.0659939283731753]


개별 주식 변동성 σ1, σ2, σ3 구하기<br>
(개별일별등락률의 모집단표준편차) * (전체 일 수의 제곱근)

In [11]:
personal_stock_v = []
std_dev = df_returns.std()
total_days = len(df_returns)
volatility = std_dev * (total_days ** 0.5)
for value in volatility:
    personal_stock_v.append(value)

print(volatility)
print("personal_stock_v:", personal_stock_v)

삼성전자    0.284494
현대차     0.373270
카카오     0.336604
dtype: float64
personal_stock_v: [0.2844937147549378, 0.3732704621271418, 0.33660384866281146]


3종목간의 상관계수 p12, p13, p23 구하기<br>
3 종목의 1년치 종가를 가져와서 3 종목간의 각각의 상관계수 구하기

In [12]:
print(selected_companies_names)
df_close_price

['삼성전자', '현대차', '카카오']


,삼성전자,현대차,카카오
Date,,,
2023-11-06,70900,177900,44700
2023-11-07,70900,176100,43750
2023-11-08,69900,174800,43950
2023-11-09,70300,174700,45600
2023-11-10,70500,173700,45650
...,...,...,...
2024-10-31,59200,215000,36950
2024-11-01,58300,213500,36150
2024-11-04,58700,216500,37350


In [13]:
# 각 종목 간의 상관계수 계산
p12_correlation = df_close_price[selected_companies_names[0]].corr(df_close_price[selected_companies_names[1]])
p13_correlation = df_close_price[selected_companies_names[0]].corr(df_close_price[selected_companies_names[2]])
p23_correlation = df_close_price[selected_companies_names[1]].corr(df_close_price[selected_companies_names[2]])

# 결과 출력
print(f"1,2번 종목의 상관계수: {p12_correlation}")
print(f"1,3번 종목의 상관계수: {p13_correlation}")
print(f"2,3번 종목의 상관계수: {p23_correlation}")

1,2번 종목의 상관계수: 0.3119744224871348
1,3번 종목의 상관계수: 0.2699410304980517
2,3번 종목의 상관계수: -0.4424130897264056


In [14]:
stock_corr = []
stock_corr.append(p12_correlation)
stock_corr.append(p13_correlation)
stock_corr.append(p23_correlation)
stock_corr

[np.float64(0.3119744224871348),
 np.float64(0.2699410304980517),
 np.float64(-0.4424130897264056)]

원래 총 조합 5151개 중 최대 0.7, 최소 0.1 조건을 만족하는 조합만 추출

In [15]:
def find_combinations():
    possible_values = [i / 100 for i in range(101)] # 0부터 1까지 0.01 간격으로 조합 추출
    combinations = itertools.product(possible_values, repeat=3)

    valid_combinations = []
    valid_indices = []

    for i, combo in enumerate(combinations):
        if round(sum(combo), 2) == 1.0 and all(0.1 <= x <= 0.7 for x in combo): # 최대 0.7, 최소 0.1은 가지는 조합만 추출
            valid_combinations.append(combo)
            valid_indices.append(i)

    return valid_combinations, valid_indices

result, valid_indices = find_combinations()

print("조합의 수:", len(result))
print("일부 예시 조합:", result[:10])

조합의 수: 2391
일부 예시 조합: [(0.1, 0.2, 0.7), (0.1, 0.21, 0.69), (0.1, 0.22, 0.68), (0.1, 0.23, 0.67), (0.1, 0.24, 0.66), (0.1, 0.25, 0.65), (0.1, 0.26, 0.64), (0.1, 0.27, 0.63), (0.1, 0.28, 0.62), (0.1, 0.29, 0.61)]


포트수익률 E(rt) 구하기<br>
(x1 x E(r1) + x2 x E(r2) + x3 x E(r3))

In [16]:
port_returns = []

for combo in result:
    port_return = sum(x * er for x, er in zip(combo, daily_stock_average_d))
    port_returns.append(port_return)

print("포트수익률 개수:", len(port_returns))
print(port_returns[:10])

포트수익률 개수: 2391
[-0.012870593010087167, -0.009775439934457855, -0.0066802868588285436, -0.003585133783199225, -0.0004899807075699134, 0.0026051723680594052, 0.005700325443688724, 0.008795478519318042, 0.011890631594947368, 0.014985784670576659]


사용자에게 날짜에 맞는 3년 국채를 입력받기



In [41]:
from pykrx import bond

# 사용자에게 날짜를 입력받음
date = input("날짜를 'YYYYMMDD' 형식으로 입력하세요: ")

# 입력받은 날짜로 데이터프레임 생성
df = bond.get_otc_treasury_yields(date)

# 국고채 3년의 수익률 데이터를 소수점 둘째 자리까지 변환하여 추출
kukko_3year_yield = df.loc['국고채 3년', '수익률'] / 100
formatted_yield = f"{kukko_3year_yield:.3f}"  # 소수점 셋째 자리까지 출력
print(f"무위험이자율: {formatted_yield}")

# 변환된 수익률을 입력 받음
government_bond = formatted_yield

날짜를 'YYYYMMDD' 형식으로 입력하세요:  20241106


무위험이자율: 0.030


σ(t) 값 구하기<br>
(x1 ^ 2 x σ1 ^ 2) + (x2 ^ 2 x σ2 ^ 2) + (x3 ^ 2 x σ3 ^ 2) + (2x1x2σ1σ2p12) + (2x2x3σ2σ3p23) + (2x1x3σ1σ3p13)

In [42]:
port_vol = []

for combo in result:
    term1 = sum(x ** 2 * sigma ** 2 for x, sigma in zip(combo, personal_stock_v))
    term2 = 0

    for i in range(len(combo) - 1):
        for j in range(i + 1, len(combo)):
            idx = i * (len(combo) - 1) + j - 1
            corr_idx = min(idx, 2)
            term2 += 2 * combo[i] * combo[j] * personal_stock_v[i] * personal_stock_v[j] * stock_corr[corr_idx]

    port_volatility = (term1 + term2) ** 0.5
    port_vol.append(port_volatility)

print("포트폴리오 변동성의 개수:", len(port_vol))
print(port_vol[:10])

포트폴리오 변동성의 개수: 2391
[np.float64(0.2264521325232856), np.float64(0.22303975997625688), np.float64(0.2197399872635096), np.float64(0.21655796162054827), np.float64(0.21349894788102544), np.float64(0.21056830724907594), np.float64(0.20777147194828297), np.float64(0.2051139155682751), np.float64(0.20260111902534328), np.float64(0.20023853217418366)]


Cal 기울기 구하기<br>
(E(rt)-Rf)/σ(t)<br>
위에서 조건을 만족하는 (최대 0.7, 최소 0.1은 배정) 조합 중에 최대 Cal 기울기 값을 가져오도록

In [43]:
# government_bond = Rf
Rf = float(government_bond)
Cal_values = []

# port_returns_numeric = [float(val) for val in port_returns]

for i in range(len(port_returns)):
    Cal = (port_returns[i] - Rf) / port_vol[i]
    Cal_values.append(Cal)

print("Cal 값의 개수:", len(Cal_values))
print(Cal_values[:10])

Cal 값의 개수: 2391
[np.float64(-0.1893141501137282), np.float64(-0.17833340539234818), np.float64(-0.1669258623139992), np.float64(-0.15508611889341165), np.float64(-0.14281091785314454), np.float64(-0.13009948168285337), np.float64(-0.11695385477347815), np.float64(-0.10337924378233483), np.float64(-0.08938434541808893), np.float64(-0.07498164896835521)]


Max Cal 기울기<br>
최대 기울기와 조합을 찾음

In [44]:
max_cal_value = max(Cal_values)
max_cal_index = Cal_values.index(max_cal_value)
max_result = result[max_cal_index]

print("Cal 기울기 최대값 : ", max_cal_value)
print("Max Cal 위치 : ", max_cal_index)
print("Max Cal 조합 : ", max_result)

Cal 기울기 최대값 :  0.443238124717785
Max Cal 위치 :  50
Max Cal 조합 :  (0.1, 0.7, 0.2)


위험비율, 투자금액 입력받기
->부동소수점 문제 때문에 int로 받고 반올림하는 식으로

In [45]:
Investment = int(input(f"투자하실 금액을 입력하세요: "))
print("투자비율을 입력하세요.")
no_danger = float(input(f"무위험 투자비율: "))
high_danger = float(input(f"고위험 투자비율: "))

invest_amount = []
real_invest = (Investment * (high_danger * 0.1))
rf = Investment - real_invest
for i in range(len(max_result)):
  price = real_invest * max_result[i]
  invest_amount.append(price)

print("-------------------------------------")
print("총 투자 금액 : {:,.0f}".format(Investment))
print("rf : {:,.0f}".format(rf))
for i in range(len(selected_companies_names)):
  print("{} 투자 금액: {:,.0f}".format(selected_companies_names[i], invest_amount[i]))

투자하실 금액을 입력하세요:  10000000


투자비율을 입력하세요.


무위험 투자비율:  7
고위험 투자비율:  3


-------------------------------------
총 투자 금액 : 10,000,000
rf : 7,000,000
삼성전자 투자 금액: 300,000
현대차 투자 금액: 2,100,000
카카오 투자 금액: 600,000
